# Narrative GMM Turning Points (銀河鉄道 × 不確実性 × GMM)

このノートブックは、物語JSON（イベント + 原文引用付き）から **(m=Morality, iso=Isolation)** の2次元点列を取り出し、
半教師ありGMM（EM）で

- 状態クラスタ（意味付き）
- 境界点Top20（揺れ：不確実性が高い点）
- 章ごとの転調密度（turning density）
- クラスタ遷移（状態切替）

を **図＋CSV** として出力します。

## Outputs（`outputs/`）
- `events_all.csv`
- `preds_all.csv`
- `cluster_name_map.csv`
- `boundary_top20.csv`
- `boundary_context.csv`
- `scene_turning_density.csv`
- `scene_cluster_transitions.csv`
- `cluster_transition_points.csv`
- `gmm_params.npz`


In [ ]:
# ===== 0) Setup =====
# (Colab) 初回だけ必要なら実行：
# !pip -q install -r requirements.txt

from pathlib import Path
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# repoルートで実行する想定：src を import path に追加
ROOT = Path.cwd()
if (ROOT / "src").exists():
    sys.path.insert(0, str(ROOT / "src"))

from narrative_gmm.io_json import load_events_df
from narrative_gmm.gmm_semisup import fit_gmm_semisup
from narrative_gmm.metrics import add_posteriors, boundary_topN, boundary_context
from narrative_gmm.report import auto_name_clusters, compute_scene_turning_density, compute_scene_transitions, compute_global_transition_points
from narrative_gmm import plot as plot_mod

JSON_PATH = "data/sample/sample_g_1_reconstructed_keep_last9.json"
OUT_DIR = Path("outputs"); OUT_DIR.mkdir(parents=True, exist_ok=True)

FEATURES = ["m", "iso"]
K = 2
ALPHA = 20.0
REG = 1e-3
DIAG_COV = True

# 再現性（教材向け）：固定すると毎回ほぼ同じ境界Top20になります
SEED = 229
N_INIT = 50
MAX_ITER = 400
TOL = 1e-4

TOP_N_BOUNDARY = 20
CONTEXT_W = 2
print("JSON_PATH:", JSON_PATH)


In [ ]:
# ===== 1) Load JSON -> DataFrame =====
# 物語（イベント/引用）を保持したまま、数値特徴(m, iso)を取り出して表にします。
df = load_events_df(JSON_PATH)
df.to_csv(OUT_DIR/"events_all.csv", index=False, encoding="utf-8-sig")
df.head(5)


In [ ]:
# ===== 2) Fit semi-supervised GMM (EM) =====
# labeled点（アンカー）を alpha 倍の強さで混ぜて、クラスタの「意味」を安定化します。
X_all = df[FEATURES].to_numpy()
mask_l = df["label"].to_numpy() >= 0
X_l = X_all[mask_l]
y_l = df.loc[mask_l, "label"].to_numpy().astype(int)
X_u = X_all[~mask_l]

ss = fit_gmm_semisup(
    X_u, X_l, y_l,
    K=K, alpha=ALPHA, reg=REG, diag=DIAG_COV,
    seed=SEED, n_init=N_INIT, max_iter=MAX_ITER, tol=TOL
)

np.savez(OUT_DIR/"gmm_params.npz", phi=ss["phi"], mu=ss["mu"], Sigma=ss["Sigma"])
print("fitted:", {k: (v.shape if hasattr(v,'shape') else v) for k,v in ss.items() if k!='it'}, "it=", ss["it"])


In [ ]:
# ===== 3) Predict + boundary extraction =====
# posterior（所属確率）から、entropy / margin を作り「揺れ」を抽出します。
preds, w_all = add_posteriors(df, X_all, ss["phi"], ss["mu"], ss["Sigma"])

# (重要) 0/1 のクラスタ番号を、人間のラベル(Communal vs Self_Protection)に寄せて自動命名
preds, name_map = auto_name_clusters(preds, K=K)
name_map.to_csv(OUT_DIR/"cluster_name_map.csv", index=False, encoding="utf-8-sig")

preds.to_csv(OUT_DIR/"preds_all.csv", index=False, encoding="utf-8-sig")

boundary = boundary_topN(preds, top_n=TOP_N_BOUNDARY)
boundary.to_csv(OUT_DIR/"boundary_top20.csv", index=False, encoding="utf-8-sig")

ctx = boundary_context(preds, boundary, context_w=CONTEXT_W)
ctx.to_csv(OUT_DIR/"boundary_context.csv", index=False, encoding="utf-8-sig")

name_map


In [ ]:
# ===== 4) Chapter summaries (転調密度 / 遷移) =====
boundary_steps = set(boundary["global_step"].astype(int).tolist())

scene_density = compute_scene_turning_density(preds, boundary_steps)
scene_density.to_csv(OUT_DIR/"scene_turning_density.csv", index=False, encoding="utf-8-sig")

scene_transitions = compute_scene_transitions(preds)
scene_transitions.to_csv(OUT_DIR/"scene_cluster_transitions.csv", index=False, encoding="utf-8-sig")

transition_points = compute_global_transition_points(preds)
transition_points.to_csv(OUT_DIR/"cluster_transition_points.csv", index=False, encoding="utf-8-sig")

scene_density.head(10)


In [ ]:
# ===== 5) Plots (decision regions なしで読みやすく) =====
plot_mod.scatter_entropy(preds, boundary, title="Clusters (size=entropy, circles=boundary Top20)")
plot_mod.centers_ellipses(preds, ss["mu"], ss["Sigma"])
plot_mod.time_series_entropy_and_clusters(preds, use_cluster_name=True)
plot_mod.chapter_bars(scene_density, scene_transitions)

# 境界Top20（本文へ還元しやすい表示）
show_cols = ["global_step","scene_id","chapter_title","m","iso","cluster_name","p_max","margin","entropy","event","evidence_1","evidence_2"]
boundary[show_cols]


## メモ：SEEDについて
- `SEED` を固定すると、初期化が同じになり **教材として再現可能** になります。
- `SEED=None` にすると、探索的に毎回違う結果が出ます（その代わり `N_INIT` を増やすと安定します）。
